# Tutorial: Generating Type Ia SN lightcurves based on ztf_sim output 

This notebook shows how to load the output for Eric's survey simulator `ztf_sim` and generate SN Ia lightcurves using the SALT2 template. (Check out the other notebooks for examples how to simulate other transients.)

*Note:* You need to download Eric's newest sample output [here](https://drive.google.com/file/d/1sB6r21ALG7ZKetvE734JS-UpLa3iwWSz/view). The link was also included in Eric's email, so you will likely only need to change the path below.

Furthermore you'll require the dust map from Schlegel, Finkbeiner & Davis (1998) for full functionality. It can be found [here](https://github.com/kbarbary/sfddata/archive/master.tar.gz).

In [1]:
# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx #
# xxxxxxxxxxxxxxxxxxxxxxx--------------Generating Type Ia Light Curves----------------------xxxxxxxxxxxxxxxxxxxxxxxxx #
# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx #

import time
t1 = time.time()
# ------------------------------------------------------------------------------------------------------------------- #
# Import Modules
# ------------------------------------------------------------------------------------------------------------------- #
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sncosmo
import simsurvey
import simsurvey_tools as sst
from astropy.cosmology import Planck15
from astropy.table import Table, vstack, hstack
# ------------------------------------------------------------------------------------------------------------------- #


# ------------------------------------------------------------------------------------------------------------------- #
# Initialize Directories
# ------------------------------------------------------------------------------------------------------------------- #
os.environ["HOME"] = "/data/asingh/simsurvey"
DIR_HOME = os.environ.get("HOME")
DIR_DATA = "/data/cfremling/simsurvey"

# Enter the name of the 'ztf_sim' output file you would like to use.
# survey_file = os.path.join(DIR_HOME, "data/test_schedule_v6.db")
survey_file = os.path.join(DIR_DATA, "notebooks/df_sim_stats_full.p")
survey_fields = os.path.join(DIR_DATA, "data/ZTF_Fields.txt")

# Enter the name of the dust map files of Schlegel, Finkbeiner & Davis (1998)
sfd98_dir = os.path.join(DIR_HOME, "data/sfd98")
# ------------------------------------------------------------------------------------------------------------------- #


# ------------------------------------------------------------------------------------------------------------------- #
# Load ZTF Fields, CCD Corners and Filters
# ------------------------------------------------------------------------------------------------------------------- #
raw_fields = np.genfromtxt(survey_fields, comments='%')
fields = {'field_id': np.array(raw_fields[:,0], dtype=int), 'ra': raw_fields[:,1], 'dec': raw_fields[:,2]}

ccds = sst.load_ztf_ccds()
sst.load_ztf_filters()
# ------------------------------------------------------------------------------------------------------------------- #

In [2]:
def mod_df(inp_df):
    """
    Modify the input Pandas DataFrame to be passed to the SimSurvey.
    Args:
    inp_df   : Input DataFrame to be modified
    Returns:
    out_df   : Output DataFrame with relevant modifications
    """
    band_dict = {1: 'ztfg', 2: 'ztfr', 3: 'ztfi'}
    
    out_df = inp_df.copy()
    out_df['filterid'] = out_df['filterid'].apply(lambda band: band_dict[band])
    out_df['skynoise'] = [(10 ** (-0.4 * (limmag - 30))) / 5. for limmag in out_df['limMag']]
    print ('Survey pointings for All ZTF Programs: {0}'.format(len(out_df)))
    print ('Survey pointings for MSIP Programs: {0}'.format(len(out_df[out_df['progid'] == 1])))
#     out_df = out_df[out_df['fieldid'] < 881]                           # For using only main grid data
    return out_df


In [10]:
# ------------------------------------------------------------------------------------------------------------------- #
# Read the Raw ZTF Input Data
# ------------------------------------------------------------------------------------------------------------------- #
raw_df = pd.read_pickle(os.path.join(DIR_DATA, survey_file)).iloc[0:-1]
raw_df = mod_df(raw_df)
# ------------------------------------------------------------------------------------------------------------------- #

Survey pointings for All ZTF Programs: 5338604
Survey pointings for MSIP Programs: 5338604


In [11]:
# ------------------------------------------------------------------------------------------------------------------- #
# Load Simulated Survey from file (Download from ftp://ftp.astro.caltech.edu/users/ebellm/one_year_sim_incomplete.db)
# Currently, DES filters are used as proxies for ZTF filters
# ------------------------------------------------------------------------------------------------------------------- #
plan = simsurvey.SurveyPlan(time=raw_df['jd'], band=raw_df['filterid'], obs_field=raw_df['fieldid'], 
                            skynoise=raw_df['skynoise'], obs_ccd=raw_df['chid'], ccds=ccds, comment=raw_df['progid'], 
                            fields={k: v for k, v in fields.items() if k in ['ra', 'dec', 'field_id']})

mjd_range = (plan.cadence['time'].min() - 30, plan.cadence['time'].max() + 30)
# ------------------------------------------------------------------------------------------------------------------- #

IndexError: index -999999999 is out of bounds for axis 0 with size 1774

In [ ]:
# ------------------------------------------------------------------------------------------------------------------- #
# Review the pointing schedule, you can use this table
# ------------------------------------------------------------------------------------------------------------------- #
print (type(plan.pointings), len(plan.pointings))
# print (ccds)
plan.pointings
# ------------------------------------------------------------------------------------------------------------------- #

## TransientGenerator
The transient generator combines a model and a distribution representing the transient population, and randomly draws all parameters needed to simulate the lightcurves. For well studied transient types, e.g. SNe Ia, models and generators have been predefined for easy use.

Here the maximum redshift has been kept very low in order make the simulation short. In reality $z_{max} = 0.2$ would be more realistic.

In [ ]:
# ------------------------------------------------------------------------------------------------------------------- #
# Define the parameters for the Transient Generator
# ------------------------------------------------------------------------------------------------------------------- #
tr = simsurvey.get_transient_generator(zrange=(0.0, 0.05), dec_range=(-31,90), ra_range=(0, 360),
                                       transient='Ia', template='salt2', mjd_range=(mjd_range[0], mjd_range[1]),
                                       sfd98_dir=sfd98_dir)
print (type(tr))
# ------------------------------------------------------------------------------------------------------------------- #

## SimulSurvey
Lastly, all parts are combined in a SimulSurvey object that will generate the lightcurves.
(This may take about a minute or two.)

In [ ]:
# ------------------------------------------------------------------------------------------------------------------- #
# Generate The Light Curves
# ------------------------------------------------------------------------------------------------------------------- #
survey = simsurvey.SimulSurvey(generator=tr, plan=plan)
lcs = survey.get_lightcurves(progress_bar=True, notebook=True)
#, notebook=True # If you get an error because of the progress_bar, delete this line.)
# ------------------------------------------------------------------------------------------------------------------- #

In [ ]:
# ------------------------------------------------------------------------------------------------------------------- #
# Check the Generated Light Curves
# ------------------------------------------------------------------------------------------------------------------- #
print (len(lcs.lcs))
print (lcs[0])

print("Time Taken = {0}".format(time.time() - t1))
# ------------------------------------------------------------------------------------------------------------------- #

## Analysing the output

The output of `get_lightcurves()` is a `LightcurveCollection` object. Lightcurves are automatically filter, so only those that would be detected in the survey are kept.

You can save a the lightcurves in a pickle file and load them again later without rerunning the simulation.

In [ ]:
# ------------------------------------------------------------------------------------------------------------------- #
# Save the Light Curves in a Pickle File
# ------------------------------------------------------------------------------------------------------------------- #
lcs.save('lcs_tutorial.pkl')
lcs = simsurvey.LightcurveCollection(load='lcs_tutorial.pkl')
# ------------------------------------------------------------------------------------------------------------------- #

You can inspect the lightcurves manually. This example should return the lightcurve with the most points with S/N > 5.

In [ ]:
_ = sncosmo.plot_lc(lcs[2])

The two figures below show how early the MNe are detected and at what redshifts. The simulation input parameters of transients that were not detected are also kept, so can check completeness. 

In [ ]:
plt.figure(figsize=(9, 7))
plt.hist(lcs.stats['p_det'], lw=2, histtype='step', range=(-20,0), bins=20)
plt.xlabel('Detection phase (observer-frame)', fontsize='x-large')
_ = plt.ylabel(r'$N_{SNe}$', fontsize='x-large')

In [ ]:
plt.figure(figsize=(9, 6))
plt.hist(lcs.meta_full['z'], lw=1, histtype='step', range=(0,0.05), bins=20, label='all')
plt.hist(lcs.meta['z'], lw=2, histtype='step', range=(0,0.05), bins=20, label='detected')
plt.xlabel('Redshift', fontsize='x-large')
plt.ylabel(r'$N_{SNe}$', fontsize='x-large')
plt.xlim((0, 0.05))
plt.legend()

In [ ]:
plan = simsurvey.SurveyPlan(load_opsim=survey_file, band_dict={'g': 'ztfg', 'r': 'ztfr', 'i': 'desi'}, ccds=ccds)
plan.pointings